## 1. Defined the LangGraph `State`

In [1]:
from typing import TypedDict

class GuideState(TypedDict):
    chunk: str
    summary: str
    concept: str
    question: str
    analogy: str
    faq: str
    study_guide: str

# 2. Defining the Nodes

In [2]:
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage

# mistral-saba-24b
# llama-3.3-70b-versatile
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=1)

## Summarize Node

In [3]:
def summarize_node(state: GuideState) -> GuideState:
    prompt = f"""
    You are an expert academic summarizer helping students quickly understand complex material. Your task is to summarize the following text chunk clearly and concisely in 2–3 sentences.

    Follow these principles:
    - Focus only on the **core concepts**, **key points**, and **essential arguments**
    - Keep the original **intent**, but **simplify the language** to aid understanding
    - Remove repetition, tangents, or overly detailed explanations
    - If a definition, statistic, or example is central, include it briefly
    - Structure your summary logically so it flows and stands on its own

    Text:
    {state['chunk']}
    """
    response = llm([HumanMessage(content=prompt)])
    return {
        **state,
        "summary": response.content.strip()
    } 

## Concepts Node 

In [4]:
def concept_node(state: GuideState) -> GuideState:
    prompt = f"""
    You are an expert knowledge mapper. Extract and explain the key concepts from the following academic text chunk.

    Follow these guidelines:
    - Identify the most important concepts, principles, theories, or terms
    - For each concept:
    - Provide a clear, concise definition  
    - Explain its role or importance in the topic  
    - Mention any relationships with other concepts (e.g., cause-effect, hierarchy)  
    - Include formulas, models, or frameworks if discussed
    - If there are debates or alternative views, mention them briefly
    - Format: Use headings for each concept followed by a short explanation (2–4 sentences)

    Text:
    {state['chunk']}

    Output: A structured list of key concepts with clear explanations.
    """ 
    response = llm([HumanMessage(content=prompt)])
    return {
        **state,
        "concept": response.content.strip()
    }

## Review questions Node

In [5]:
def review_question_node(state: GuideState) -> GuideState:
    prompt = f"""
    You are an expert educational assessment designer. Create thoughtful review questions based on the following academic text chunk.

    Guidelines:
    - Generate 3 to 5 questions that test understanding of core ideas and concepts
    - Use a variety of question types:
    - Recall (basic understanding)
    - Application (using ideas in new contexts)
    - Analysis (exploring relationships between concepts)
    - Evaluation (critical thinking or critique)
    - Each question should be clear, focused, and meaningful
    - For every question, provide a detailed answer explanation
    - Avoid yes/no or overly simplistic questions
    - Encourage deeper learning and reflection

    Text:
    {state["chunk"]}

    Output: A list of review questions with comprehensive answer explanations.
    """

    response = llm([HumanMessage(content=prompt)])
    return {
        **state,
        "question": response.content.strip()
    }

## Analogy Node

In [6]:
def analogy_node(state: GuideState) -> GuideState:
    prompt = f"""
    You are an expert in making complex concepts accessible. Create illuminating analogies for the key concepts in the provided text chunk.

    Follow these guidelines:
    - Identify 2-3 challenging or abstract concepts from the text
    - For each concept, create an analogy that:
    * Uses familiar, everyday scenarios or objects
    * Accurately represents the key relationships or properties of the concept
    * Helps bridge the gap between the unfamiliar and the familiar
    - Explain how the analogy maps to the original concept
    - Note any limitations of the analogy (where it breaks down)
    - Ensure analogies are culturally inclusive and widely accessible
    - When possible, use diverse domains for analogies (e.g., sports, cooking, nature)

    Input:
    {state["chunk"]}

    Output: Helpful analogies that make complex concepts more intuitive and memorable.
    """
 
    response = llm([HumanMessage(content=prompt)])
    return {
            **state,
            "analogy": response.content.strip()
        } 

## FAQs node

In [7]:
def FAQs_node(state: GuideState) -> GuideState:
    prompt = f"""
    You are an expert educational content developer who specializes in creating helpful FAQ sections. Generate a comprehensive FAQ section based on the provided text chunk that resembles those found in high-quality reference books.

    Follow these guidelines:
    - Identify 4-6 questions that students or readers genuinely struggle with when learning this material
    - Create questions that:
    * Address common misconceptions or points of confusion
    * Cover practical applications of theoretical concepts
    * Clarify boundaries between similar or easily confused concepts
    * Address "why" questions that explain underlying principles or rationale
    * Include questions about exceptions to rules or special cases
    - Format questions as natural, conversational inquiries a student might actually ask
    - Craft detailed, clear answers that:
    * Directly address the core confusion
    * Provide illuminating examples where helpful
    * Reference specific sections or content from the original material
    * Anticipate and address follow-up questions
    - Include at least one question that addresses how this content connects to broader themes or topics
    - Ensure the FAQ section feels like a genuine resource, not a quiz or assessment

    Input: {state["chunk"]}

    Output: An authentic FAQ section that addresses real learning challenges with thoughtful, illuminating responses.
    """
    response = llm([HumanMessage(content=prompt)])
    return {
        **state,
        "faq": response.content.strip()
    }

## Study Guide Node 

In [8]:
def integrative_guide_node(state: GuideState) -> GuideState:
    prompt = f"""
    You are an expert educational consultant designing personalized study plans. Integrate the information from the document chunks to create an optimal, intelligent study guide.

    Follow these guidelines:
    - Review all information from the summarization, concept mapping, review questions, and analogies
    - Create a cohesive study plan that:
    * Presents a logical learning progression from foundational to advanced concepts
    * Identifies the 3-5 most critical areas to focus on based on complexity and importance
    * Suggests specific study techniques suited to the material (e.g., spaced repetition, concept mapping)
    * Recommends time allocation for different topics based on difficulty and significance
    - Include metacognitive prompts that encourage the student to reflect on their understanding
    - Suggest connections to prior knowledge or related fields when relevant
    - Provide a brief "quick reference" section for essential formulas, definitions, or principles
    - Adapt language and approach based on apparent complexity of the material

    Input:
    Summary:
    {state["summary"]}

    Concepts:
    {state["concept"]}

    Review Questions:
    {state["question"]}

    Analogies:
    {state["analogy"]}

    FAQs:
    {state["faq"]}
    Output: A personalized, intelligent study guide that maximizes understanding and retention while being genuinely helpful and accessible.
    """
 
    response = llm([HumanMessage(content=prompt)])
    return {
        **state,
        "study_guide": response.content.strip()
    }

In [9]:
from langgraph.graph import StateGraph, END
from langchain.schema.runnable import RunnableLambda

summarize = RunnableLambda(summarize_node)
concept = RunnableLambda(concept_node)
review_qs = RunnableLambda(review_question_node)
analogy = RunnableLambda(analogy_node)
faq = RunnableLambda(FAQs_node)
study_guide = RunnableLambda(integrative_guide_node)

# 3. Defined the Graph

In [10]:
builder = StateGraph(GuideState)

builder.add_node("summarize_node", summarize)
builder.add_node("extract_concepts_node", concept)
builder.add_node("generate_review_qs_node", review_qs)
builder.add_node("generate_analogies_node", analogy)
builder.add_node("build_FAQs_node", faq)
builder.add_node("build_study_guide_node", study_guide)

builder.set_entry_point("summarize_node")
builder.add_edge("summarize_node", "extract_concepts_node")
builder.add_edge("extract_concepts_node", "generate_review_qs_node")
builder.add_edge("generate_review_qs_node", "generate_analogies_node")
builder.add_edge("generate_analogies_node", "build_FAQs_node")
builder.add_edge("build_FAQs_node", "build_study_guide_node")
builder.set_finish_point("build_study_guide_node")

In [11]:
graph = builder.compile()

# Creating the dummy data for testing

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

import os

# 1. Sample markdown note (as a string)
markdown_text = """
# Machine Learning Basics

Machine learning is a field of artificial intelligence that enables computers to learn from data without being explicitly programmed. It is commonly used in applications such as image recognition, speech processing, and predictive analytics.

## Types of Machine Learning

- **Supervised Learning:** Learns a function from labeled data.
- **Unsupervised Learning:** Finds patterns from unlabeled data.
- **Reinforcement Learning:** Learns by trial and error, receiving rewards or penalties.

## Common Algorithms

- Linear Regression
- Decision Trees
- Support Vector Machines
- Neural Networks
"""

# 2. Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = text_splitter.create_documents([markdown_text])

# 3. Create Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Store in ChromaDB
db = Chroma.from_documents(documents=docs, embedding=embedding_model, persist_directory="data/chroma_db")
db.persist()  # Save to disk

# 5. Query it!
query = "What is supervised learning?"
results = db.similarity_search(query, k=2)

# Show results
for i, res in enumerate(results):
    print(f"\n🔹 Result {i+1}:")
    print(res.page_content)


/tmp/ipykernel_38287/2002993475.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/vivek/miniforge3/envs/NCvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔹 Result 1:
## Types of Machine Learning

- **Supervised Learning:** Learns a function from labeled data.
- **Unsupervised Learning:** Finds patterns from unlabeled data.
- **Reinforcement Learning:** Learns by trial and error, receiving rewards or penalties.

## Common Algorithms

🔹 Result 2:
## Types of Machine Learning

- **Supervised Learning:** Learns a function from labeled data.
- **Unsupervised Learning:** Finds patterns from unlabeled data.
- **Reinforcement Learning:** Learns by trial and error, receiving rewards or penalties.

## Common Algorithms


/tmp/ipykernel_38287/2002993475.py:36: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()  # Save to disk


In [13]:
results = db.get()
chunk = results["documents"]

In [14]:
chunk

['# Machine Learning Basics\n\nMachine learning is a field of artificial intelligence that enables computers to learn from data without being explicitly programmed. It is commonly used in applications such as image recognition, speech processing, and predictive analytics.\n\n## Types of Machine Learning',
 '## Types of Machine Learning\n\n- **Supervised Learning:** Learns a function from labeled data.\n- **Unsupervised Learning:** Finds patterns from unlabeled data.\n- **Reinforcement Learning:** Learns by trial and error, receiving rewards or penalties.\n\n## Common Algorithms',
 '## Common Algorithms\n\n- Linear Regression\n- Decision Trees\n- Support Vector Machines\n- Neural Networks',
 '# Machine Learning Basics\n\nMachine learning is a field of artificial intelligence that enables computers to learn from data without being explicitly programmed. It is commonly used in applications such as image recognition, speech processing, and predictive analytics.\n\n## Types of Machine Learn

In [15]:
input_state = {"chunk": chunk}

output_state = graph.invoke(input_state)

from pprint import pprint
pprint(output_state)

/tmp/ipykernel_38287/652054920.py:15: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt)])


{'analogy': '### Analogies for Machine Learning Concepts\n'
            '\n'
            "To make complex machine learning concepts more accessible, let's "
            'create illuminating analogies for supervised learning, '
            'unsupervised learning, and reinforcement learning.\n'
            '\n'
            '#### 1. Supervised Learning\n'
            '\n'
            '**Analogy:** Cooking with a Recipe\n'
            '\n'
            'Imagine you want to learn how to make your favorite dish. In '
            'supervised learning, the "recipe" represents the labeled data, '
            'and the "dish" is the function or relationship the model learns. '
            'Just as a recipe guides you through the steps to prepare a meal, '
            'labeled data guides the machine learning model to learn the '
            'patterns and relationships between inputs and desired outputs. '
            'The "chef" (the model) learns to make the "dish" (predict the '
            'out

In [17]:
a = dict(output_state)
print(a["study_guide"])


**Optimal Study Guide: Machine Learning**

**Introduction:**
Welcome to the world of machine learning, a field of artificial intelligence that enables computers to learn from data without explicit programming. This study guide is designed to provide a logical learning progression, focusing on the most critical areas, and suggesting specific study techniques to ensure a deep understanding of machine learning concepts.

**Critical Areas to Focus On:**

1. **Types of Machine Learning**: Understand the differences between supervised, unsupervised, and reinforcement learning, and how they are applied in various contexts.
2. **Common Machine Learning Algorithms**: Familiarize yourself with linear regression, decision trees, support vector machines, and neural networks, and learn how to choose the right algorithm for a given problem.
3. **Data Preprocessing and Feature Engineering**: Learn how to prepare and transform data for machine learning algorithms, including data cleaning, feature scal